# Notebook for utforsking av tekstlengder

In [ ]:
import pandas as pd
import plotly.express as px
from langchain_text_splitters import RecursiveCharacterTextSplitter

from nks_kbs_analyse.knowledgebase import load, split_documents

In [ ]:
# Hent dokumentene fra BigQuery
docs = load()

data = []
# Endre hvert item fra langchain Documents til dict
for doc in docs:
    doc_data = {"page_content": doc.page_content}
    doc_data.update(doc.metadata)
    data.append(doc_data)

# Konverter listen til en pandas DataFrame
df = pd.DataFrame(data)

In [ ]:
# Antall tegn per artikkelseksjon
df["char_count"] = df["page_content"].apply(len)

## Tekstlengder på artikkelnivå

In [ ]:
# For å finne total artikkellengde summeres seksjonslengdene per KnowledgeArticleId
df_artikkel = df.groupby("KnowledgeArticleId").agg({"char_count": "sum"}).reset_index()

In [ ]:
df_artikkel["char_count"].describe()

In [ ]:
fig = px.histogram(
    df_artikkel,
    x="char_count",
    nbins=100,
    title="Fordeling for antall tegn per artikkel",
)
fig.update_layout(xaxis_title="Antall tegn", yaxis_title="Antall artikler")
fig.show()

In [ ]:
fig = px.ecdf(
    df_artikkel,
    x="char_count",
    title="Kumulativ fordeling for antall tegn per artikkel",
)
fig.update_layout(xaxis_title="Antall tegn", yaxis_title="Andel av artikler")
fig.show()

In [ ]:
# Av nysgjerrighet - hva er outlieren med over 100K tegn?
df_artikkel[df_artikkel["char_count"] > 100000]

## Tekstlengder på seksjonsnivå

In [ ]:
df["char_count"].describe()

In [ ]:
fig = px.histogram(
    df, x="char_count", nbins=100, title="Fordeling for antall tegn per artikkelseksjon"
)
fig.update_layout(xaxis_title="Antall tegn", yaxis_title="Antall artikkelseksjoner")
fig.show()

In [ ]:
fig = px.ecdf(
    df, x="char_count", title="Kumulativ fordeling for antall tegn per artikkelseksjon"
)
fig.update_layout(xaxis_title="Antall tegn", yaxis_title="Andel av artikkelseksjoner")
fig.show()

### Seksjonslengder gruppert etter kolonne

In [ ]:
fig = px.histogram(
    df,
    x="char_count",
    facet_col="ContentColumn",
    facet_col_wrap=3,
    histnorm="percent",
    title="Fordelinger for antall tegn per artikkelseksjon gruppert",
)

fig.update_layout(height=1000)
fig.show()

In [ ]:
fig = px.ecdf(
    df,
    x="char_count",
    facet_col="ContentColumn",
    facet_col_wrap=3,
    title="Kumulativ fordeling for antall tegn per artikkelseksjon gruppert",
)
fig.update_layout(height=1000)
fig.show()

## Paragrafnivå

Hvis vi definerer at en ny paragraf starter etter dobbel linjeskift...

In [ ]:
df["paragrafer"] = df["page_content"].str.split("\n\n", expand=False)

In [ ]:
def find_pragraph_lengths(text):
    """Hjelpefunksjon for å finne lengde per paragraf."""
    paragraphs = text.split("\n\n")
    return [len(paragraph) for paragraph in paragraphs]


df["paragraph_lengths"] = df["page_content"].apply(find_pragraph_lengths)

In [ ]:
# Flatten the lists in the 'paragraph_lengths' column
flattened_lengths = [
    length for sublist in df["paragraph_lengths"] for length in sublist
]

# Create a new DataFrame from the flattened list
flattened_df = pd.DataFrame({"paragraph_lengths": flattened_lengths})

In [ ]:
fig = px.histogram(
    flattened_df, x="paragraph_lengths", title="Fordeling for antall tegn per paragraf"
)
fig.show()

In [ ]:
fig = px.ecdf(
    flattened_df,
    x="paragraph_lengths",
    title="Kumulativ fordeling for antall tegn per paragraf",
)
fig.show()

## Lengder per dokument slik vi splitter dem i dag

In [ ]:
docs = list(load())
split_docs = split_documents(list(docs), chunk_size=1500, overlap=100)
len(split_docs)

In [ ]:
flattened_split_docs = pd.DataFrame(
    {"split_doc_lengths": [len(doc.page_content) for doc in split_docs]}
)

In [ ]:
fig = px.histogram(
    flattened_split_docs,
    x="split_doc_lengths",
    title="Fordeling for antall tegn per splittet dokument",
)
fig.update_layout(
    xaxis_title="Antall tegn", yaxis_title="Antall dokumenter(etter splitting)"
)
fig.show()

In [ ]:
fig = px.ecdf(
    flattened_split_docs,
    x="split_doc_lengths",
    title="Kumulativ fordeling for antall tegn per splittet dokument",
)
fig.update_layout(
    xaxis_title="Antall tegn", yaxis_title="Andel av dokumentene (etter splitting)"
)
fig.show()

In [ ]:
# Inspiserer innholdet i de korteste dokumentene
[doc.page_content for doc in split_docs if len(doc.page_content) < 50]

## Utforske andre konfigurasjoner av splitteren

In [ ]:
# Tar bort markdown language
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)

In [ ]:
# Inspiserer innholdet i de korteste dokumentene på nytt
[doc.page_content for doc in split_docs if len(doc.page_content) < 50]